In [ ]:
import numpy as np
from scipy.stats import pearsonr
import datetime
import pandas as pd
import glob
import matplotlib.pyplot as plt
from matplotlib import gridspec
import datetime
from regrid import regrid_fast
# from regrid import regrid
import seaborn as sns
import xarray as xr
import os
import tqdm
from netCDF4 import Dataset
import warnings
warnings.filterwarnings("ignore")

In [ ]:
datapath = '/Users/carmennab/Dropbox/OI_PolarSnow/data/ERA5/snowfall/'
EASE_lats = np.load('/Users/carmennab/Dropbox/OI_PolarSnow/EASE/auxiliary/new_lat_25km.npy')
EASE_lons = np.load('/Users/carmennab/Dropbox/OI_PolarSnow/EASE/auxiliary/new_lon_25km.npy')
days = np.arange(0,4018)
dates = [datetime.date(2010,1,1)+datetime.timedelta(days=int(days)) for days in days]
all_sfs = np.full((len(days),721,1440),np.nan)

# load daily snowfall into giant array
for day in tqdm.tqdm(days): 
    try:
        date = dates[day].strftime('%Y%m%d')
        file = datapath + 'daily-sf_' + date + '.nc'
        data = Dataset(file,'r')
        sf = data['sf']  
        all_sfs[day] = sf[0]   
    
    except FileNotFoundError:
        pass
    
lon = data['longitude']
lat = data['latitude']
era5_lon,era5_lat = np.meshgrid(np.array(lon),np.array(lat))
    
# regrid onto EASE 2.0 grid
all_sf = regrid_fast(all_sfs, era5_lon, era5_lat, EASE_lons, EASE_lats)

In [ ]:
# create snowfall NETCDF
days = np.arange(0,4018)
data_vars = {'Snowfall':(['t','x','y'],all_sf)}
ds = xr.Dataset(data_vars = data_vars,
                coords={'Longitude':(['x','y'],EASE_lons),
                        'Latitude':(['x','y'],EASE_lats),
                         'Day':(['t'],days)}     )
ds.to_netcdf(f'/Users/carmennab/Dropbox/OI_PolarSnow/EASE/data/ERA5_snowfall.nc','w')